In [2]:
sample_in1 = """inp x foo
mul x -1 far daole"""
sample_in2 = """inp z
inp x
mul z 3
eql z x"""
sample_in3 = """inp w
add z w
mod z 2
div w 2
add y w
mod y 2
div w 2
add x w
mod x 2
div w 2
mod w 2"""


def get_input(s):
    return [instr.split(' ')[:3] for instr in s.strip().split('\n')]


print(get_input(sample_in1))


[['inp', 'x', 'foo'], ['mul', 'x', '-1']]


In [ ]:
def zero(s):
    return 0


def parse(instr, data, pending_code):
    code = instr[0]
    var1 = instr[1]
    if code == 'inp':
        #data[var1] = lambda s: int(s[pending_code])
        data[var1] = f'int(s[{pending_code}])'
        return pending_code + 1
    if code == 'mul':
        #data[var1] = lambda s: data.get(var1, zero)(s) * get_val(data, instr[2])(s)
        if instr[2] == '0':
            data[var1] = '0'
        elif instr[2] == '1':
            pass
        else:
            data[var1] = '(' + data[var1] + ')*(' + \
                get_valparse(data, instr[2]) + ')'
        return pending_code
    if code == 'add':
        #data[var1] = lambda s: data.get(var1, zero)(s) + get_val(data, instr[2])(s)
        if instr[2] == '0':
            pass
        else:
            data[var1] = '' + data[var1] + '+' + \
                get_valparse(data, instr[2]) + ''
        return pending_code
    if code == 'div':
        #data[var1] = lambda s: data.get(var1, zero)(s) // get_val(data, instr[2])(s)
        if instr[2] == '1':
            pass
        else:
            data[var1] = '(' + data[var1] + ')//(' + \
                get_valparse(data, instr[2]) + ')'
        return pending_code
    if code == 'mod':
        # data[var1] = lambda s: data.get(var1, zero)( s) % get_val(data, instr[2])(s)
        data[var1] = '(' + data[var1] + ')%(' + \
            get_valparse(data, instr[2]) + ')'
        return pending_code
    if code == 'eql':
        # f1 = data.get(var1, zero)
        # f2 = get_val(data, instr[2])
        # data[var1] = lambda s: 1 if f1(s) == f2(s) else 0
        data[var1] = '1 if (' + data[var1] + ')==(' + \
            get_valparse(data, instr[2]) + ') else 0'
        return pending_code
    assert False, instr


def get_valparse(data, int_or_var):
    if int_or_var in ['w', 'x', 'y', 'z']:
        return data[int_or_var]
    else:
        return int_or_var


def get_val(data, int_or_var):
    if int_or_var in ['w', 'x', 'y', 'z']:
        return lambda s: data.get(int_or_var, zero)(s)
    else:
        return lambda s: int(int_or_var)


def simulate_code(instructions, input_code):
    parsed_data = {c: '0' for c in "wxyz"}
    ic = 0
    for instr in instructions:
        ic = parse(instr, parsed_data, ic)
    print(parsed_data)
    s = input_code
    print(f'x evals to {eval(parsed_data["x"])}')
    return eval(parsed_data['z'])


def simulate_parsed(parsed, input_code):
    s = input_code
    print(s)
    return eval(parsed['z'])


def test_prob1(s):
    r = simulate_code(get_input(s), '723456789')
    print(r)
    return r


assert test_prob1(sample_in1) == 0
assert test_prob1(sample_in2) == 0
assert test_prob1(sample_in3) == 1

PARSED = {}


In [ ]:
import tqdm
import re


def prob1(s):
    global PARSED
    test_code = '9' * 14
    instructions = get_input(s)
    parsed_data = {c: '0' for c in "wxyz"}
    ic = 0
    for instr in tqdm.tqdm(instructions):
        ic = parse(instr, parsed_data, ic)
    print(f'lenght before cleanup {len(parsed_data["z"])}')
    parsed_data['z'] = cleanup(parsed_data['z'])
    print(f'lenght after cleanup {len(parsed_data["z"])}')
    PARSED = parsed_data
    while True:
        r = simulate_parsed(parsed_data, test_code)
        if r['z'] == 0:
            return test_code
        test_code = get_next(test_code)


def get_next(s):
    for i in range(13, -1, -1):
        if s[i] > '1':
            return s[0:i] + str(int(s[i])-1) + ('9' * (13 - i))


def cleanup(s):
    sold = ''
    while sold != s:
        sold = s
        s = re.sub(r'\(0\)', '0', s)
        s = re.sub(r'\(0\*[^()]*\)', '0', s)
        s = re.sub(r'\(0\*\([^(]*\)\)', '0', s)
        s = re.sub(r'\(\([^()]*\)\*0\)', '0', s)
        s = re.sub(r'\([^()]*\*0\)', '0', s)
    return s


assert cleanup('(0)') == '0'
assert cleanup('(0*1234)') == '0'
assert cleanup('(12345*0)') == '0'

assert get_next('9' * 14) == ('9' * 13) + '8', get_next('9' * 14)
assert get_next('9' * 13 + '1') == '9' * 12 + '89'
assert get_next('2' + '1'*13) == '1' + '9' * 13
assert get_next('12' + '1' * 12) == '11' + '9'*12

if False:
    prob1(my_in)


In [ ]:
import tqdm


def get_val_int(data, int_or_var):
    if int_or_var in ['w', 'x', 'y', 'z']:
        return data.get(int_or_var, 0)
    else:
        return int(int_or_var)


def run_step(instr, data, pending_code):
    code = instr[0]
    var1 = instr[1]
    if code == 'inp':
        data[var1] = int(pending_code[0])
        return pending_code[1:]
    if code == 'mul':
        data[var1] = data[var1] * get_val_int(data, instr[2])
        return pending_code
    if code == 'add':
        data[var1] = data[var1] + get_val_int(data, instr[2])
        return pending_code
    if code == 'div':
        data[var1] = data[var1] // get_val_int(data, instr[2])
        return pending_code
    if code == 'mod':
        data[var1] = data[var1] % get_val_int(data, instr[2])
        return pending_code
    if code == 'eql':
        data[var1] = 1 if data[var1] == get_val_int(data, instr[2]) else 0
        return pending_code
    assert False, instr


def simulate_code(instructions, input_code):
    parsed_data = {c: 0 for c in "wxyz"}
    ic = input_code
    for instr in instructions:
        ic = run_step(instr, parsed_data, ic)
    return parsed_data


def prob1(s):
    instructions = get_input(s)
    test_code = '9' * 14
    for i in tqdm.tqdm(range(99999999999999)):
        r = simulate_code(instructions, test_code)
        if r['z'] == 0:
            return test_code
        if int(test_code) % 100 == 1:
            print(test_code, r['z'])
        test_code = get_next(test_code)


prob1(my_in)


In [ ]:
def parse_tree(instr, data, in_index):
    code = instr[0]
    var1 = instr[1]
    if code == 'inp':
        data[var1] = ('i', in_index)
        return in_index + 1
    if code == 'mul':
        data[var1] = reduce_tree('*', data[var1], get_val_tree(data, instr[2]))
        return in_index
    if code == 'add':
        data[var1] = reduce_tree('+', data[var1], get_val_tree(data, instr[2]))
        return in_index
    if code == 'div':
        data[var1] = reduce_tree('/', data[var1], get_val_tree(data, instr[2]))
        return in_index
    if code == 'mod':
        data[var1] = reduce_tree('%', data[var1], get_val_tree(data, instr[2]))
        return in_index
    if code == 'eql':
        data[var1] = reduce_tree('=', data[var1], get_val_tree(data, instr[2]))
        return in_index
    assert False, instr


_data = {'x': (1, 2, 3), 'y': 1}
parse_tree(('mul', 'x', '0'), data=_data, in_index='')
assert _data == {'x': 0, 'y': 1}, _data


def get_val_tree(data, int_or_tree):
    if int_or_tree in 'wxyz':
        return data[int_or_tree]
    else:
        return int(int_or_tree)


def reduce_tree(instr, left, right):
    if instr == '*' and (right == 0 or left == 0):
        return 0
    if isinstance(right, int) and isinstance(left, int):
        return apply_tree((instr, left, right), '')
    return (instr, left, right)


def apply_tree(tree, input_code):
    if isinstance(tree, int):
        return tree
    code = tree[0]
    if code == 'i':
        return int(input_code[tree[1]])
    code, left, right = tree
    left_val = apply_tree(left, input_code)
    right_val = apply_tree(right, input_code)
    if code == '*':
        return left_val * right_val
    if code == '+':
        return left_val + right_val
    if code == '/':
        return left_val // right_val
    if code == '%':
        return left_val % right_val
    if code == '=':
        return 1 if left_val == right_val else 0
    assert False, code


def print_tree(t, levels=3):
    if levels < 0:
        print('...', end='')
        return
    if isinstance(t, int):
        print(t, end='')
    elif t[0] == 'i':
        print('in', t[1], end='')
    else:
        code, left, right = t
        print('(', end='')
        print_tree(left, levels-1)
        print(code, end='')
        print_tree(right, levels-1)
        print(')', end='')


CACHE = {}


def prob1tree(s):
    global CACHE
    instructions = get_input(s)
    data = {i: 0 for i in 'wxyz'}
    i = 0
    for instr in tqdm.tqdm(instructions):
        i = parse_tree(instr, data, i)
    print('finished parsing')
    CACHE = data
    print('w')
    print_tree(data['w'])
    print('x')
    print_tree(data['x'])
    print('y')
    print_tree(data['y'])
    print('z')
    print_tree(data['z'])
    test_code = '9' * 14
    for i in tqdm.tqdm(range(99999999999999)):
        r = apply_tree(data['z'], test_code)
        if r == 0:
            return test_code
        if int(test_code) % 100 == 1:
            print(test_code, r['z'])
        test_code = get_next(test_code)


prob1tree(my_in)


In [7]:
import tqdm


def get_input(s):
    return [instr.split(' ')[:3] for instr in s.strip().split('\n')]


def key_to_dict(k):
    w, x, y, z = k
    return {'w': w, 'x': x, 'y': y, 'z': z, }


def dict_to_key(k):
    return (k['w'], k['x'], k['y'], k['z'])


def append_to_dict(data, datakey, datavalues):
    new_key = dict_to_key(datakey)
    if new_key not in data:
        data[new_key] = set()
    data[new_key] = datavalues
    return


def get_val_int(data, int_or_name):
    if int_or_name in 'wxyz':
        return int(data[int_or_name])
    return int(int_or_name)


def apply_instr_split(instr, data):
    code = instr[0]
    var1 = instr[1]
    new_data = {}
    print_data = False
    for k, values in data.items():
        dk = key_to_dict(k)
        if code == 'inp':  # split
            for i in range(1, 10):
                dk[var1] = i
                new_val = values + str(i)
                append_to_dict(new_data, dk, new_val)
            print_data = True
        elif code == 'mul':
            dk[var1] = dk[var1] * get_val_int(dk, instr[2])
            append_to_dict(new_data, dk, values)
        elif code == 'add':
            dk[var1] = dk[var1] + get_val_int(dk, instr[2])
            append_to_dict(new_data, dk, values)
        elif code == 'div':
            dk[var1] = dk[var1] // get_val_int(dk, instr[2])
            append_to_dict(new_data, dk, values)
        elif code == 'mod':
            dk[var1] = dk[var1] % get_val_int(dk, instr[2])
            append_to_dict(new_data, dk, values)
        elif code == 'eql':
            dk[var1] = 1 if dk[var1] == get_val_int(dk, instr[2]) else 0
            append_to_dict(new_data, dk, values)
        else:
            assert False, instr
    if print_data:
        print(len(new_data))
    return new_data


def prob1(s):
    instructions = get_input(s)
    data = {(0, 0, 0, 0): ''}
    for instr in tqdm.tqdm(instructions):
        data = apply_instr_split(instr, data)
        if len(data) > 10_000:
            break
    print(f'Got {len(data)} options')
    solutions = {}
    for k, values in tqdm.tqdm(data.items()):
        if k[3] == 0:
            solutions.add(values)
    print(f'Found {len(solutions)} solutions')
    ss = list(sorted(solutions))
    print(ss[:2], ss[-2:])


prob1(my_in)


 24%|██▍       | 60/252 [00:00<00:00, 526.16it/s]

9
81
729
6561
7290


100%|██████████| 65610/65610 [00:00<00:00, 2010243.66it/s]

65610
Got 65610 options
Found 0 solutions
[] []


In [2]:
my_in = """
inp w  ======0
mul x 0 x=0 --------cleanup x
add x z x=0
mod x 26 x=0
div z 1 z=0
add x 13 x=13
eql x w x=0
eql x 0 x=1
mul y 0 y=0 --------cleanup y
add y 25 y=25 x=1
mul y x  y=25 x=1
add y 1  y=26 x=1
mul z y z=0 y=26 x=1
mul y 0 y=0 --------cleanup y
add y w y=w=s[0]
add y 14 y=s[0]+14
mul y x y=s[0]+14 x=1 z=0
add z y z=y=s[0]+14 x=1
inp w   ======1
mul x 0 x=0 --------cleanup x
add x z 0 x=s[0]+14
mod x 26 x=(s[0]+14) % 26
div z 1 z=s[0]+14
add x 12 x=12+(s[0] + 14)
eql x w x=0
eql x 0 x=1
mul y 0 y=0 --------cleanup y
add y 25 y=25 x=1 z=s[0]+14
mul y x y=25
add y 1 y=26
mul z y z=26*(s[0]+14)
mul y 0 y=0 --------cleanup y
add y w y=s[1]
add y 8 y=s[1]+8
mul y x y=s[1]+8
add z y z=s[1]+8 + s[0]+14
inp w   =====2
mul x 0 x=0 --------cleanup x
add x z x=s[1]+8 + s[0]+14
mod x 26 x=(s[1]+8 + s[0]+14) % 26
div z 1  z=s[1]+8 + s[0]+14
add x 11 x=11 + random
eql x w  x=0
eql x 0  x=1
mul y 0 y=0 --------cleanup y
add y 25  y=25
mul y x  y=25
add y 1  y=26
mul z y  z=26 * (s[1]+8 + s[0]+14)
mul y 0 y=0 --------cleanup y
add y w y=s[2]
add y 5 y=s[2]+5
mul y x y=s[2]+5
add z y z=26 * (s[1]+8 + s[0]+14) + s[2] + 5
inp w   =======3
mul x 0 x=0 --------cleanup x
add x z x =26 * (s[1]+8 + s[0]+14) + s[2] + 5 
mod x 26 x = s[2] + 5
div z 26 z = 26*(s[1]+8 + s[0]+14)//26 + (s[2] + 5)//26 
add x 0
eql x w  x = 1 if s[3] == s[2]+5 else 0
eql x 0  x = 0 if s[3] == s[2]+5 else 1
mul y 0 y=0 --------cleanup y
add y 25  y=25
mul y x  y = 0 if s[3] == s[2]+5 else 25
add y 1  y = 1 if s[3] == s[2]+5 else 26
mul z y  z = s[1]+s[0]+22 if s[3] == s[2]+5 else s[1]+s[0]+48
mul y 0 y=0 --------cleanup y
add y w y=s[3]
add y 4 y=s[3]+4
mul y x y= 0 if s[3]==s[2]+5 else s[3]+4
add z y z=s[1]+s[0]+22 if s[3] == s[2]+5 else s[3]+4+s[1]+s[0]+48
inp w   =========4
mul x 0 x=0 --------cleanup x
add x z x=z=s[1]+s[0]+22 if s[3] == s[2]+5 else s[3]+s[1]+s[0]+52
mod x 26 x=(s[1]+s[0]+22) % 26 if s[3]==s[2]+5 else (s[3]+s[1]+s[0]) % 26
div z 1
add x 15
eql x w x=0 because x>15 and w must be between 1 and 9
eql x 0 x=1
mul y 0 y=0 --------cleanup y
add y 25 y=25 x=1 z=s[1]+s[0]+22 if s[3] == s[2]+5 else s[3]+s[1]+s[0]+52
mul y x
add y 1 y=26
mul z y z=26*(s[1]+s[0]+22) if s[3] == s[2]+5 else 26*(s[3]+s[1]+s[0]+52)
mul y 0 y=0 --------cleanup y
add y w y=s[4]
add y 10 y=s[4]+10
mul y x y=s[4]+10
add z y z=(s[4]+10)*26*(s[1]+s[0]+22) if s[3] == s[2]+5 else (s[3]+s[1]+s[0]+52)*26*(s[4]+10)
inp w   ==========5
mul x 0 x=0 --------cleanup x
add x z x=z=(s[4]+10)*26*(s[1]+s[0]+22) if s[3] == s[2]+5 else (s[3]+s[1]+s[0]+52)*26*(s[4]+10)
mod x 26 x=0
div z 26 z=(s[4]+10)*(s[1]+s[0]+22) if s[3] == s[2]+5 else (s[3]+s[1]+s[0]+52)*(s[4]+10)
add x -13 x=-13
eql x w x=0
eql x 0 x=1
mul y 0 y=0 --------cleanup y
add y 25 y=25
mul y x y=25
add y 1 y=26
mul z y z=z=(s[4]+10)*26*(s[1]+s[0]+22) if s[3] == s[2]+5 else (s[3]+s[1]+s[0]+52)*26*(s[4]+10)
mul y 0 y=0 --------cleanup y
add y w y=s[5]
add y 13 y=s[5] + 13
mul y x y=y
add z y z=s[5] + 13 + (s[4]+10)*26*(s[1]+s[0]+22) if s[3] == s[2]+5 else (s[3]+s[1]+s[0]+52)*26*(s[4]+10) + s[5] + 13
inp w   =========6
mul x 0 x=0 --------cleanup x
add x z  x=err
mod x 26  x=err
div z 1  z=z
add x 10 x>9
eql x w  x=0
eql x 0  x=1
mul y 0 y=0 --------cleanup y
add y 25 y=25
mul y x  y=25
add y 1  y=26 x=1 w=s[6] z=s[5] + 13 + (s[4]+10)*26*(s[1]+s[0]+22) if s[3] == s[2]+5 else (s[3]+s[1]+s[0]+52)*26*(s[4]+10) + s[5] + 13
mul z y  z=26*s[5] +26*13 + (s[4]+10)*26*26*(s[1]+s[0]+22) if s[3]==s[2]+5 else 26*26*(s[3]+s[1]+s[0]+52)*(s[4]+10) + 26*s[5] + 26*13
mul y 0 y=0 --------cleanup y
add y w  y=s[6]
add y 16 y=s[6]+16
mul y x  y=s[6]+16  x=1
add z y  z=s[6] + 16 + 26*s[5] +26*13 + (s[4]+10)*26*26*(s[1]+s[0]+22) if s[3]==s[2]+5 else 26*26*(s[3]+s[1]+s[0]+52)*(s[4]+10) + 26*s[5] + 26*13 + s[6]+16
inp w    ==========7
mul x 0 x=0 --------cleanup x
add x z  x=z
mod x 26 x=s[6] + 16
div z 26 z=
add x -9
eql x w
eql x 0
mul y 0 y=0 --------cleanup y
add y 25
mul y x
add y 1
mul z y
mul y 0 y=0 --------cleanup y
add y w
add y 5
mul y x
add z y
inp w
mul x 0 x=0 --------cleanup x
add x z
mod x 26
div z 1
add x 11
eql x w
eql x 0
mul y 0 y=0 --------cleanup y
add y 25
mul y x
add y 1
mul z y
mul y 0 y=0 --------cleanup y
add y w
add y 6
mul y x
add z y
inp w
mul x 0 x=0 --------cleanup x
add x z
mod x 26
div z 1
add x 13
eql x w
eql x 0
mul y 0 y=0 --------cleanup y
add y 25
mul y x
add y 1
mul z y
mul y 0 y=0 --------cleanup y
add y w
add y 13
mul y x
add z y
inp w
mul x 0 x=0 --------cleanup x
add x z
mod x 26
div z 26
add x -14
eql x w
eql x 0
mul y 0 y=0 --------cleanup y
add y 25
mul y x
add y 1
mul z y
mul y 0 y=0 --------cleanup y
add y w
add y 6
mul y x
add z y
inp w
mul x 0 x=0 --------cleanup x
add x z
mod x 26
div z 26
add x -3
eql x w
eql x 0
mul y 0 y=0 --------cleanup y
add y 25
mul y x
add y 1
mul z y
mul y 0 y=0 --------cleanup y
add y w
add y 7
mul y x
add z y
inp w
mul x 0 x=0 ---------- cleanup x
add x z
mod x 26
div z 26
add x -2
eql x w
eql x 0
mul y 0 y=0 --------cleanup y
add y 25
mul y x
add y 1
mul z y
mul y 0 y=0 ------- cleanup y
add y w
add y 13
mul y x
add z y
inp w   w=s[14]
mul x 0 x=0  -------- cleanup x
add x z x=x+z=z
mod x 26 x=x%26=z%26
div z 26 z=z/26
add x -14 x=x-14
eql x w x=(x==w)
eql x 0 x=(x==0)=(x'<>w)
mul y 0 y=0 y=0 --------cleanup y
add y 25 y=25
mul y x y=y*x=25*x
add y 1 y=y+1=25*x+1
mul z y z=z*y=z*(25+x+1)
mul y 0 y=0 --------cleanup y
add y w y=y+w=w
add y 3 y=y+3=w+3
mul y x y=y*x=(w+3)*x
add z y z+y=z+(w+3)*x
"""


In [ ]:

def depth_tree(t):
    if isinstance(t, int):
        return 1
    if t[0] == 'i':
        return 1
    return max(depth_tree(t[1]), depth_tree(t[2])) + 1


depth_tree(CACHE['y'])
